In [1]:
cd /content/drive/MyDrive/NLUProject

/content/drive/MyDrive/NLUProject


In [2]:
!pip install pytorch_lightning
!pip install transformers
!pip install pytorch-lightning-bolts
!pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade

     |████████████████████████████████| 582 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
     |████████████████████████████████| 136 kB 14.2 MB/s 
     |████████████████████████████████| 408 kB 49.2 MB/s 
     |████████████████████████████████| 1.1 MB 9.5 MB/s 
     |████████████████████████████████| 271 kB 34.8 MB/s 
     |████████████████████████████████| 144 kB 7.3 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 77 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
     |████████████████████████████████| 252 kB 3.7 MB/s 
  Cloning https://github.com/PytorchLightning/lightning-bolts.git (to revision master) to /tmp/pip-req-bui

In [3]:
from pl_classifier import NLU_dataset, NLU_DataModule, NLU_Toxic_classifier
import pickle
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
from pl_bolts.callbacks import PrintTableMetricsCallback
from sklearn.preprocessing import MultiLabelBinarizer


import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt
import re
import pandas as pd

import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AdamW, BertConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
from transformers import RobertaConfig, RobertaModel
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
import numpy as np
import random
import time
import datetime

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def get_metrics(label, pred):
    label = list(label)
    pred = list(pred)
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(pred)): 
        if label[i]==1 and pred[i]==1:
            TP += 1
        if pred[i]==1 and label[i]!=pred[i]:
            FP += 1
        if label[i]==0 and pred[i]==0:
            TN += 1
        if pred[i]==0 and label[i]!=pred[i]:
            FN += 1

    return(TP, FP, TN, FN)
  
def metrics_print(data,label_col, pred_col):

    data_male = data[data.Gender == 0]
    data_female = data[data.Gender == 1]
    TP_M, FP_M, TN_M, FN_M = get_metrics(data_male[label_col], data_male[pred_col] )
    TP_F, FP_F, TN_F, FN_F = get_metrics(data_female[label_col], data_female[pred_col] )
    F1_M = f1_score(data_male[label_col], data_male[pred_col])
    F1_F = f1_score(data_female[label_col], data_female[pred_col])
    
    print(f"Overall Accuracy: {accuracy_score(data.Label, data[pred_col])}")
    print(f"Male Accuracy: {accuracy_score(data_male.Label, data_male[pred_col])}")
    print(f"Female Accuracy: {accuracy_score(data_female.Label, data_female[pred_col])}")
    print(f"False Positive Equality Difference {np.abs((FP_M/(FP_M+TN_M)) - (FP_F/(FP_F+TN_F)))}")
    print(f"False Negative Equality Difference {np.abs((FN_M/(FN_M+TP_M)) - (FN_F/(FN_F+TP_F)))}")
    print(f"Disparity Score {np.abs(F1_M-F1_F)}")


In [4]:
data = pd.read_csv("/content/drive/MyDrive/NLUProject/Data/SentimentAnalysisData/Detox_Dataset")
val = pd.read_pickle("/content/drive/MyDrive/NLUProject/Data/SentimentAnalysisData/val.pkl")
train = pd.read_pickle("/content/drive/MyDrive/NLUProject/Data/SentimentAnalysisData/train.pkl")
test = pd.read_pickle("/content/drive/MyDrive/NLUProject/Data/SentimentAnalysisData/test.pkl")

val = val[val.Updated==1]
train = train[train.Updated==1]
test = test[test.Updated==1]

#parameters
N_EPOCHS = 3
BATCH_SIZE = 12
MAX_LEN = 256
LR = 2e-05
opt_thresh = 0.4

directory = '/content/drive/MyDrive/NLUProject/'
mlb = MultiLabelBinarizer()

val['Label'] = mlb.fit_transform(val.Label.astype(str)).tolist()
train['Label'] = mlb.fit_transform(train.Label.astype(str)).tolist()
test['Label'] = mlb.fit_transform(test.Label.astype(str)).tolist()

steps_per_epoch= data.shape[0] // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
special_tokens_dict = {'additional_special_tokens': ['[TOXIC]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

#Training Model
new_model = NLU_Toxic_classifier(n_classes=2,
                              n_warmup_steps=warmup_steps,
                              n_training_steps=total_training_steps)
data_module = NLU_DataModule(
  train,
  val,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_LEN)
data_module.setup()

trainer = pl.Trainer(max_epochs=N_EPOCHS, gpus=1, progress_bar_refresh_rate=3)

trainer.fit(new_model, data_module)

trainer.save_checkpoint("/content/drive/MyDrive/NLUProject/DeBERTa_220407_6.ckpt")



Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533M [00:00<?, ?B/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
#Inference  
new_model = NLU_Toxic_classifier.load_from_checkpoint(
            checkpoint_path="/content/drive/MyDrive/NLUProject/DeBERTa_220407_6.ckpt",
            n_classes=2)

data_module = NLU_DataModule(
  test,
  test,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_LEN)

data_module.setup()

trainer = pl.Trainer(max_epochs=N_EPOCHS, gpus=1, progress_bar_refresh_rate=3)

prd = trainer.predict(new_model, data_module)


Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForMaskedLM: ['lm_predictions.lm_head.dense.weight', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.bias', 'cls.predicti

Predicting: 0it [00:00, ?it/s]

In [ ]:
predicted = pd.Series(prd).apply(lambda x : torch.argmax(x[1],axis=1).tolist()).apply(pd.Series).stack().reset_index(drop=True)
test['pred'] = predicted.values.astype(int)
test['Label'] = test.Label.apply(np.argmax)

In [ ]:
metrics_print(test, 'Label', 'pred')

Overall Accuracy: 0.8153034300791556
Male Accuracy: 0.8091993185689949
Female Accuracy: 0.8231740801757276
False Positive Equality Difference 0.024852271763408637
False Negative Equality Difference 0.045362300585223225
Disparity Score 0.03500911777652005
